In [1]:
import pandas as pd
import operator
import pymongo

import numpy as np

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.charged_up

# Declare the collection
charging_stations = db.charging_stations
electric_vehicles = db.electric_vehicles



In [3]:
charging_stations

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'charged_up'), 'charging_stations')

In [4]:
rows = [row for row in charging_stations.find()]

In [5]:
rows[1]['address']['country']

'South Korea'

In [6]:
count = 1
stations = []
country = []
city = {}

for k,row in enumerate(charging_stations.find()):
      #print(row)
            if row['address']['country'] == 'USA':
                #stations.append({count:row})
                
                temp_total_count = 0   
                if row['address']['city'] in city:                
                    
                    #print(row['address']['city'])
                    
                    temp_total_count = int(city[row['address']['city']])

                    temp_total_count += 1

                    city[row['address']['city']] = str(temp_total_count)
                else:
                    city[row['address']['city']] = "1"

sorted_US_data = sorted(city.items(), key=operator.itemgetter(1))
city_df = pd.DataFrame(sorted_US_data).rename(columns={0:'City',1:'Charging Stations'})\
            .sort_values('Charging Stations',ascending=False)
city_df.head(10)

,City,Charging Stations
953,San Jose,9
952,San Diego,8
951,Madison,8
950,Chicago,7
949,Las Vegas,7
948,Miami,6
945,Richmond,5
946,Springfield,5
947,Los Angeles,5
936,Sacramento,4


In [9]:
elect_df = pd.DataFrame([row for row in electric_vehicles.find()])
elect_df.head()

,_id,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,608b1bf0f0b6cb6008526909,Tesla,Model 3 Long Range Dual Motor,4,233,450,161,940.0,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,608b1bf0f0b6cb600852690a,Volkswagen,ID.3 Pure,10,160,270,167,250.0,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,608b1bf0f0b6cb600852690b,Polestar,2,4,210,400,181,620.0,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,608b1bf0f0b6cb600852690c,BMW,iX3,6,180,360,206,560.0,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,608b1bf0f0b6cb600852690d,Honda,e,9,145,170,168,190.0,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


In [10]:
elect_df.keys()

Index(['_id', 'Brand', 'Model', 'AccelSec', 'TopSpeed_KmH', 'Range_Km',
       'Efficiency_WhKm', 'FastCharge_KmH', 'RapidCharge', 'PowerTrain',
       'PlugType', 'BodyStyle', 'Segment', 'Seats', 'PriceEuro'],
      dtype='object')

In [12]:
grp = elect_df.groupby('Model')[['PriceEuro','TopSpeed_KmH','Range_Km']]

### 5 Most Expensive

In [17]:
grp.max().sort_values('PriceEuro',ascending=False).head()

,PriceEuro,TopSpeed_KmH,Range_Km
Model,,,
Roadster,215000,410,970
Taycan Turbo S,180781,260,375
Taycan Cross Turismo,150000,250,385
One,149000,150,575
Taycan Turbo,148301,260,390


### 5 Fastest

In [18]:
grp.max().sort_values('TopSpeed_KmH',ascending=False).head()

,PriceEuro,TopSpeed_KmH,Range_Km
Model,,,
Roadster,215000,410,970
Model 3 Long Range Performance,61480,261,435
Model S Performance,96990,261,505
Taycan Turbo S,180781,260,375
Taycan Turbo,148301,260,390


### 5 Top Range

In [19]:
grp.max().sort_values('Range_Km',ascending=False).head()

,PriceEuro,TopSpeed_KmH,Range_Km
Model,,,
Roadster,215000,410,970
Cybertruck Tri Motor,75000,210,750
Air,105000,250,610
One,149000,150,575
Model S Long Range,79990,250,515
